# Scraping Top Repositoty for Topics on GitHub

In [2]:
# Import all required Libraries
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd

base url for scraping site

In [3]:
base_url="https://github.com/topics"

Performing follwing information

    Title of the top git repository contaning 
    1.Topic_name
    2.Topic_desription
    3.Topic url
  


Downloading the the html page

In [4]:
import requests
from bs4 import BeautifulSoup

def get_topics_page():
    
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = bs4(response.text, 'html.parser')
    return doc

In [5]:
doc = get_topics_page()

Function for obtaining topic title

In [6]:
def get_topic_titles(doc):
    
    topic_title_tags = doc.find_all("p",{'class':'f3 lh-condensed mb-0 mt-1 Link--primary'})
    topic_titles = []
    for i in range(len(topic_title_tags)):
        topic_titles.append(topic_title_tags[i].text)
    return topic_titles

In [7]:
titles=get_topic_titles(doc)

In [8]:
len(titles)

30

In [9]:
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

In [10]:
def get_topic_url(doc):
    selection_class = "no-underline flex-grow-0"
    topic_url_tags = doc.find_all("a",{'class':"no-underline flex-grow-0"})
    topic_url = []
    base_url='https://github.com'
    for i in range(len(topic_url_tags)):
        topic_url.append(base_url+topic_url_tags[i]['href'])
    return topic_url

In [11]:
url=get_topic_url(doc)

In [12]:
url[:3]

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm']

Topic desription

In [13]:
def get_topic_des(doc):
    selection_class = "no-underline flex-grow-0"
    topic_des_tags = doc.find_all("p",{'class':"f5 color-fg-muted mb-0 mt-1"})
    topic_des = []
    
    for i in range(len(topic_des_tags)):
        topic_des.append(topic_des_tags[i].text.strip())
    return topic_des

In [14]:
des=get_topic_des(doc)

In [15]:
des[:5]

['3D modeling is the process of virtually developing the surface and structure of a 3D object.',
 'Ajax is a technique for creating interactive web applications.',
 'Algorithms are self-contained sequences that carry out a variety of tasks.',
 'Amp is a non-blocking concurrency library for PHP.',
 'Android is an operating system built by Google designed for mobile devices.']

Scrap all contents into csv fle

In [16]:
def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_des(doc),
        'url': get_topic_url(doc)
    }
    return pd.DataFrame(topics_dict)

In [17]:
scraped_title=scrape_topics()

In [18]:
scraped_title[:5]

,title,description,url
0,3D,3D modeling is the process of virtually develo...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


# Get the top 20 repositories from a topic page

In [19]:
def get_topic_page(topic_url):
    # Download the page
    response = requests.get(topic_url)
    # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    # Parse using Beautiful soup
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc

In [20]:
doc = get_topic_page('https://github.com/topics/ajax')

In [21]:
def parse_star_count(stars):
    stars=stars.strip()
    if stars[-1]=='k':
        return int(float(stars[:-1])*1000)
    return(int(stars))

In [22]:
def get_repo_info(repo_tags, star_tags):  
    a=repo_tags.find_all('a')
    user_name=a[0].text.strip()
    repo_name=a[1].text.strip()
    url_repo="https://github.com/topics"+ a[1]['href']
    star_tags=parse_star_count(star_tags.text)
    return user_name, repo_name, url_repo, star_tags

In [23]:
def get_topics_page():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [24]:
def get_topic_repo(doc):
    # Get the h3 tags containing repo title, repo URL and username
    repo_tags = doc.find_all('h3',{'class':"f3 color-fg-muted text-normal lh-condensed"})

    # Get star tags
    star_tags=doc.find_all('span', {'id':'repo-stars-counter-star'})


    topic_repo_dict={ "repo_name":[],"user_name":[],"repo_url":[],"star":[]}

    for i in range(len(repo_tags)):
        repo_info=get_repo_info(repo_tags[i], star_tags[i]) #use function et_topics_page
        topic_repo_dict["repo_name"].append(repo_info[0])
        topic_repo_dict["user_name"].append(repo_info[1])
        topic_repo_dict["star"].append(repo_info[3])
        topic_repo_dict["repo_url"].append(repo_info[2])
         
    return pd.DataFrame(topic_repo_dict)
     

From git, Ajax is one of top repository with url=https://github.com/topics/ajax

In [25]:
doc = get_topic_page('https://github.com/topics/ajax')

In [26]:
scrap_content=get_topic_repo(doc)

In [27]:
len(scrap_content)

20

In [28]:
scrap_content[:5]

,repo_name,user_name,repo_url,star
0,ljianshu,Blog,https://github.com/topics/ljianshu/Blog,7200
1,metafizzy,infinite-scroll,https://github.com/topics/metafizzy/infinite-s...,7100
2,developit,unfetch,https://github.com/topics/developit/unfetch,5400
3,jquery-form,form,https://github.com/topics/jquery-form/form,5100
4,olifolkerd,tabulator,https://github.com/topics/olifolkerd/tabulator,4800
